In [298]:
from enum import Enum

# Definitions (assumed from earlier examples)
class Name(Enum):
    PETER = 1
    VALERIA = 2

class Color(Enum):
    RED = 1
    BLUE = 2

class House(Enum):
    H1 = 1  # House 1, Position 1
    H2 = 2  # House 2, Position 2
    H3 = 3

# Operator: Equality check
con =  lambda x,y : abs(x-y) == 1

# Constraint: Name.PETER MUST BE Color.RED
constraint_element = [
    con,       # [0] Operator (==)
    Name,          # [1] Category 1 (Name)
    Name.PETER,    # [2] Variable 1 (PETER)
    Color,         # [3] Category 2 (Color)
    Color.RED      # [4] Variable 2 (RED)
]

In [299]:
# The complete Domain Structure
CSP_DOMAINS = {
    # House 1 (H1)
    House.H1: {
        Name: [Name.PETER, Name.VALERIA],  # Domain for the Name category
        Color: [Color.RED, Color.BLUE]     # Domain for the Color category
    },

    # House 2 (H2)
    House.H2: {
        Name: [Name.PETER, Name.VALERIA],
        Color: [Color.RED, Color.BLUE]
    },
    House.H3:{
        Name: [Name.PETER, Name.VALERIA],
        Color: [Color.RED, Color.BLUE]
    }

}

In [300]:

def print_domains(domains):
    """
    prints the domains structure.
    """
    print("\n--- CSP-Domain ---")

    for house_entity, categories in domains.items():

        print(f"\nEntität: {house_entity.name} (Position: {house_entity.value})")

        for category_class, domain_list in categories.items():
            domain_names = [member.name for member in domain_list]

            print(f"  ▪︎  {category_class.__name__}: {domain_names}")

    print("\n--- --- ---")
print_domains(CSP_DOMAINS)


--- CSP-Domain ---

Entität: H1 (Position: 1)
  ▪︎  Name: ['PETER', 'VALERIA']
  ▪︎  Color: ['RED', 'BLUE']

Entität: H2 (Position: 2)
  ▪︎  Name: ['PETER', 'VALERIA']
  ▪︎  Color: ['RED', 'BLUE']

Entität: H3 (Position: 3)
  ▪︎  Name: ['PETER', 'VALERIA']
  ▪︎  Color: ['RED', 'BLUE']

--- --- ---


In [301]:
def revise_domain(domains, constraint, entity_a_id, entity_b_id):
    """
    Adjusts the domain of Entity A by removing all values for which
    no supporting value exists in the domain of Entity B, conditional
    on the House Operator.

    :param domains: The CSP_DOMAINS structure.
    :param constraint: The 5-element constraint array.
    :param entity_a_id: ID of the entity being revised (e.g., House.H1).
    :param entity_b_id: ID of the supporting entity (e.g., House.H2).
    :return: True if the domain of Entity A was revised (reduced), otherwise False.
    """

    # 1. Extract Constraint Parts
    house_operator = constraint[0]

    kat_a = constraint[1]
    var_a = constraint[2] # Specific value A (e.g., Name.PETER)

    kat_b = constraint[3]
    var_b = constraint[4] # Specific value B (e.g., Color.RED)

    # 2. Domain Access
    domain_a = domains[entity_a_id][kat_a]
    domain_b = domains[entity_b_id][kat_b]

    revised = False
    new_domain_a = []

    # A. CHECK HOUSE POSITIONS
    # The constraint only applies if the House Operator (e.g., equality) is True
    if house_operator(entity_a_id.value, entity_b_id.value):

        # B. DOMAIN REDUCTION (Arc Consistency Check)

        # 3. Revision: Check every value in Domain A (v_a)
        for v_a in domain_a:
            has_support = False

            # Check for Support in Domain B (v_b)
            for v_b in domain_b:

                # CONSISTENCY CHECK (Equivalence: PETER <-> RED)
                is_v_a_var = (v_a == var_a)
                is_v_b_var = (v_b == var_b)

                # Check if the boolean states match (True==True or False==False)
                # This ensures that if v_a is PETER, v_b must be RED, and vice versa.
                if is_v_a_var == is_v_b_var:
                    has_support = True
                    break  # Support found

            # 4. Update Domain based on Support
            if has_support:
                new_domain_a.append(v_a) # Keep value
            else:
                # No support: Value must be removed
                revised = True

        # 5. Replace Domain (only if the House Operator was True)
        if revised or len(new_domain_a) < len(domain_a):
            domains[entity_a_id][kat_a] = new_domain_a

        return revised

    else:
        # House Operator is False (e.g., H1 != H2)
        # Skip revision, return False (no change made)
        print("Operator is False")
        return False

In [302]:
# Initial Status:
print_domains(CSP_DOMAINS)




--- CSP-Domain ---

Entität: H1 (Position: 1)
  ▪︎  Name: ['PETER', 'VALERIA']
  ▪︎  Color: ['RED', 'BLUE']

Entität: H2 (Position: 2)
  ▪︎  Name: ['PETER', 'VALERIA']
  ▪︎  Color: ['RED', 'BLUE']

Entität: H3 (Position: 3)
  ▪︎  Name: ['PETER', 'VALERIA']
  ▪︎  Color: ['RED', 'BLUE']

--- --- ---


In [303]:
# Simulation: Red got allready deleted from house 2, so only blue is left, which should result in house 1 and 3 not having peter anymore
CSP_DOMAINS[House.H2][Color] = [Color.BLUE]

print(revise_domain(CSP_DOMAINS,constraint_element,House.H1,House.H2)) #abs 1 can constrain



True


In [304]:
print(revise_domain(CSP_DOMAINS,constraint_element,House.H1,House.H3)) #abs 2 cant constrain

Operator is False
False


In [305]:
print(revise_domain(CSP_DOMAINS,constraint_element,House.H2,House.H2)) #abs 0 cant constrain

Operator is False
False


In [306]:
print(revise_domain(CSP_DOMAINS,constraint_element,House.H3,House.H2)) #abs 1 can constrain

True


In [307]:
# after csp-ing
print_domains(CSP_DOMAINS)


--- CSP-Domain ---

Entität: H1 (Position: 1)
  ▪︎  Name: ['VALERIA']
  ▪︎  Color: ['RED', 'BLUE']

Entität: H2 (Position: 2)
  ▪︎  Name: ['PETER', 'VALERIA']
  ▪︎  Color: ['BLUE']

Entität: H3 (Position: 3)
  ▪︎  Name: ['VALERIA']
  ▪︎  Color: ['RED', 'BLUE']

--- --- ---
